In [2]:
import face_recognition
from rtree import index
import os
import numpy as np

In [6]:
import face_recognition
import random
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from sklearn.neighbors import BallTree
import heapq

# Ruta a la carpeta con las imágenes de rostros
folder_path = "/Users/camila/Library/Mobile Documents/com~apple~CloudDocs/Ciclo 5/bd2/dumb/BD2-Lab10_2/lfw"

# Obtener la lista de archivos de imagen en la carpeta
file_list = os.listdir(folder_path)

# Seleccionar N rostros aleatorios
N = 1000
print("Tamaño de la lista: ", len(file_list))
random_indices = random.sample(range(len(file_list)), N)

data = []

# Cargar las imágenes y extraer las codificaciones de rostros
for i in random_indices:
    image_path = os.path.join(folder_path, file_list[i])
    image = face_recognition.load_image_file(image_path)
    face_encoding = face_recognition.face_encodings(image)[0]
    data.append(face_encoding)

# Convertir la lista de codificaciones de rostros a un array numpy
data = np.array(data)

# Definir la función de búsqueda KNN secuencial
def knn_sequential(query_vector, data, k):
    distances = [(np.linalg.norm(query_vector - data_vector), i) for i, data_vector in enumerate(data)]
    return heapq.nsmallest(k, distances)  # Retorna los 'k' menores elementos

# Medir el tiempo de la búsqueda KNN secuencial
start_time = time.time()
knn_sequential(data[0], data, 10)
print("--- Búsqueda KNN secuencial: %s segundos ---" % (time.time() - start_time))

# Crear el BallTree para la búsqueda KNN indexada
tree = BallTree(data)

# Medir el tiempo de la búsqueda KNN indexada
start_time = time.time()
dist, ind = tree.query([data[0]], k=10)
print("--- Búsqueda KNN indexada: %s segundos ---" % (time.time() - start_time))


Tamaño de la lista:  5749


IsADirectoryError: [Errno 21] Is a directory: '/Users/camila/Library/Mobile Documents/com~apple~CloudDocs/Ciclo 5/bd2/dumb/BD2-Lab10_2/lfw/Jamie_Martin'

In [7]:
import face_recognition
import os
import numpy as np
from rtree import index
import random
import time

# Ruta a la carpeta con las imágenes de rostros
folder_path = "/Users/camila/Library/Mobile Documents/com~apple~CloudDocs/Ciclo 5/bd2/dumb/BD2-Lab10_2/lfw"

# Obtener la lista de subcarpetas y archivos en la carpeta
subfolders_files = os.listdir(folder_path)

# Crear una lista vacía para almacenar las rutas completas de las imágenes
image_paths = []

# Iterar a través de las subcarpetas y archivos
for subfolder_file in subfolders_files:
    # Crear la ruta completa al subfolder/archivo
    subfolder_file_path = os.path.join(folder_path, subfolder_file)
    
    # Si es una subcarpeta, iterar a través de los archivos dentro de la subcarpeta
    if os.path.isdir(subfolder_file_path):
        files = os.listdir(subfolder_file_path)
        for file in files:
            file_path = os.path.join(subfolder_file_path, file)
            # Agregar la ruta completa de la imagen a la lista
            image_paths.append(file_path)
    # Si es un archivo, agregar la ruta completa de la imagen a la lista
    else:
        image_paths.append(subfolder_file_path)

# Crear un RTree y una lista para almacenar los vectores de datos
p = index.Property()
p.dimension = 128  # Los vectores característicos de face_recognition tienen una dimensión de 128
idx = index.Index(properties=p)
data_vectors = []

for i, image_path in enumerate(image_paths):
    try:
        image = face_recognition.load_image_file(image_path)
        face_encodings = face_recognition.face_encodings(image)

        if len(face_encodings) > 0:
            face_encoding = face_encodings[0]

            # Agregar el vector al RTree
            idx.insert(i, tuple(face_encoding))

            # Agregar el vector a la lista
            data_vectors.append(face_encoding)
    except:
        print(f"Error processing {image_path}")

print("Finished building RTree and data_vectors.")


Finished building RTree and data_vectors.


In [8]:
import heapq

def knn_sequential(query_vector, data_vectors, k):
    heap = []

    for i, data_vector in enumerate(data_vectors):
        distance = np.linalg.norm(data_vector - query_vector)

        if len(heap) < k:
            heapq.heappush(heap, (-distance, i))
        else:
            heapq.heappushpop(heap, (-distance, i))

    indices = [i for distance, i in heap]
    return indices



In [11]:
N = 10 ** 4  # Número de consultas
k = 5  # Número de vecinos más cercanos para KNN

random_indices = random.sample(range(len(data_vectors)), N)
query_vectors = [data_vectors[i] for i in random_indices]

start = time.time()

for query_vector in query_vectors:
    knn_sequential(query_vector, data_vectors, k)

end = time.time()
sequential_time = end - start
print(f"Sequential KNN :{sequential_time} secs.")

start = time.time()

for query_vector in query_vectors:
    list(idx.nearest(tuple(query_vector), k))

end = time.time()
indexed_time = end - start
print(f"Indexed KNN : {indexed_time} seconds.")




Sequential KNN :652.2648141384125 secs.
Indexed KNN : 557.0000820159912 seconds.
